## Exercise

We have seen how to compute the log-odds between liberal-conservative for each book. Given this information, we can try to estimate political leanings of students. You can do this by summing the log-odds of their favorite books. 

Steps: 
1. Create a table with the log-odds of the books. 
2. Join the table with the log-odds with the book preferences table.
3. Sum the log-odds score for each student.

Evaluation:
* You have students that have declared their political preferences as Liberal, Conservative, Very Liberal, Very Conservative. Examine the scores for these students, to check how well this technique works. The simplest way is to compute the average (mean) log-odds for students that fall into the different groups. Alternatively, you can try to plot the full distribution of scores.
* Calculate a score for each student that did not declare a political view but has listed Favorite Books.

Notes: 
* You can do the work in MySQL or in Pandas. If you decide to work purely in Pandas, the [`merge`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html#pandas.DataFrame.merge) command allows you to perform joins between dataframes, in way similar to SQL.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (15, 7)


In [2]:
conn_string_fb = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
    user='student',
    password='dwdstudent2015',
    host='db.ipeirotis.org',
    port=3306,
    db='facebook')
engine_fb = create_engine(conn_string_fb)

books = '''
SELECT B.Book, P.PoliticalViews, COUNT(*) AS cnt 
FROM Profiles P JOIN FavoriteBooks B ON B.ProfileID = P.ProfileId  
WHERE PoliticalViews IS NOT NULL AND B.Book IS NOT NULL 
      AND (PoliticalViews = 'Liberal' OR PoliticalViews = 'Conservative')
AND B.Book IN (
    SELECT Book 
    FROM FavoriteBooks B JOIN Profiles P ON B.ProfileID = P.ProfileId  
    WHERE (P.PoliticalViews = 'Liberal' OR P.PoliticalViews = 'Conservative')
    GROUP BY Book HAVING COUNT(DISTINCT P.ProfileID)>10
)
GROUP BY B.Book, P.PoliticalViews;
'''
df_books = pd.read_sql(books, con=engine_fb)
# Fill the NaN entries with the value 0 
dfp = df_books.pivot_table(
    index='Book', 
    columns='PoliticalViews', 
    values='cnt').fillna(0)

In [3]:
polviews = '''
SELECT PoliticalViews, COUNT(*) AS cnt 
FROM facebook.Profiles
GROUP BY PoliticalViews
'''
df_polviews = pd.read_sql(polviews, con=engine_fb).set_index('PoliticalViews')
liberals = df_polviews.at['Liberal','cnt']
conservatives = df_polviews.at['Conservative','cnt']
dfp["Liberal_perc"] = 100*(dfp["Liberal"] +1)  / liberals
dfp["Conservative_perc"] = 100*(dfp["Conservative"] +1)  / conservatives

In [4]:
dfp["lift_liberal"]          = dfp["Liberal_perc"] / dfp["Conservative_perc"]
dfp["lift_conservative"]     = dfp["Conservative_perc"]  / dfp["Liberal_perc"]
dfp["log_odds_liberal"]      = np.log(dfp["lift_liberal"])
dfp["log_odds_conservative"] = np.log(dfp["lift_conservative"])

In [5]:
# Calculate the log-odds score for each student, by summing the log-odds score for the books they like
dfp

PoliticalViews,Conservative,Liberal,Liberal_perc,Conservative_perc,lift_liberal,lift_conservative,log_odds_liberal,log_odds_conservative
Book,,,,,,,,
100 Years Of Solitude,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
1984,32.0,286.0,4.442037,3.525641,1.259923,0.793699,0.231051,-0.231051
A Clockwork Orange,8.0,86.0,1.346541,0.961538,1.400402,0.714080,0.336760,-0.336760
A Confederacy Of Dunces,2.0,37.0,0.588144,0.320513,1.835010,0.544956,0.607050,-0.607050
A Doll S House,0.0,12.0,0.201207,0.106838,1.883300,0.530983,0.633025,-0.633025
A Farewell To Arms,8.0,36.0,0.572667,0.961538,0.595573,1.679054,-0.518231,0.518231
A Fine Balance,0.0,23.0,0.371460,0.106838,3.476861,0.287616,1.246130,-1.246130
A Heartbreaking Work Of Staggering Genius,8.0,70.0,1.098901,0.961538,1.142857,0.875000,0.133531,-0.133531
A Million Little Pieces,7.0,39.0,0.619099,0.854701,0.724346,1.380556,-0.322486,0.322486


In [15]:
people='''SELECT Name, Book,PoliticalViews FROM Profiles  INNER JOIN FavoriteBooks
On Profiles.ProfileID=FavoriteBooks.ProfileID
WHERE Book is not null '''
df_people = pd.read_sql(people, con=engine_fb)
df_people

,Name,Book,PoliticalViews
0,Brian Whitton,Books Are For Suckers,Libertarian
1,Sunny Kim,Mansfield Park,Conservative
2,Sunny Kim,Memoirs Of A Geisha,Conservative
3,Sunny Kim,Harry Potter All,Conservative
4,Sunny Kim,A Gesture Life,Conservative
5,Sunny Kim,Queen Of Camelot,Conservative
6,Sunny Kim,Middlesex,Conservative
7,Sunny Kim,Cat S Cradle,Conservative
8,Sunny Kim,Diary,Conservative
9,Sunny Kim,The Bible,Conservative


In [16]:
df_new=pd.merge(df_people,dfp,on="Book",how="inner",sort=True)
df_new

,Name,Book,PoliticalViews,Conservative,Liberal,Liberal_perc,Conservative_perc,lift_liberal,lift_conservative,log_odds_liberal,log_odds_conservative
0,Morayo Faleyimu,100 Years Of Solitude,Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
1,Robbie Mantel,100 Years Of Solitude,Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
2,Katie Matteson,100 Years Of Solitude,Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
3,Lauren Gage,100 Years Of Solitude,Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
4,Mark DiMilia,100 Years Of Solitude,Moderate,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
5,Sarah Burton,100 Years Of Solitude,None,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
6,Rachel Gold,100 Years Of Solitude,Conservative,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
7,Joe Ketner,100 Years Of Solitude,Very Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
8,Andrea Strane,100 Years Of Solitude,Very Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133
9,Vicky Su,100 Years Of Solitude,Liberal,3.0,55.0,0.866739,0.427350,2.028169,0.493056,0.707133,-0.707133


In [27]:
df_odds=df_new.groupby(['Name', 'PoliticalViews'],as_index=False).sum()
df_odds

,Name,PoliticalViews,Conservative,Liberal,Liberal_perc,Conservative_perc,lift_liberal,lift_conservative,log_odds_liberal,log_odds_conservative
0,A.J. Allegra,Liberal,26.0,248.0,3.869370,2.991453,2.418877,1.678478,0.365416,-0.365416
1,A.J. Puzzo,Liberal,96.0,856.0,13.310633,10.683761,4.379001,4.309332,-0.010398,0.010398
2,A.M. Dupee,Other,1.0,11.0,0.185730,0.213675,0.869215,1.150463,-0.140164,0.140164
3,ALI Morris,Other,5.0,160.0,2.507352,0.747863,5.780282,0.804148,1.972424,-1.972424
4,Aahirah Amir,Liberal,17.0,220.0,3.451478,2.136752,5.625755,1.643835,1.844351,-1.844351
5,Aaron Beim,Conservative,23.0,113.0,1.764433,2.564103,0.688129,1.453216,-0.373779,0.373779
6,Aaron Boutte,Apathetic,3.0,10.0,0.170252,0.427350,0.398390,2.510101,-0.920323,0.920323
7,Aaron Cahan,Very Liberal,0.0,60.0,0.959604,0.213675,8.981891,0.469268,2.951792,-2.951792
8,Aaron Garcia,Liberal,101.0,795.0,12.505804,12.179487,14.765741,13.239537,0.708862,-0.708862
9,Aaron Gold,Very Liberal,74.0,591.0,9.178146,8.119658,1.797772,2.510864,-0.334079,0.334079


In [34]:
# Calculate the mean log-odds score for students with different political views 
# (hint: Very Conservatives will be "out of order" compared to "Conservatives")
df_avg=df_odds.groupby(['PoliticalViews'],as_index=True).mean()
df_avg=df_avg.sort_values(by="log_odds_liberal")
df_avg

,Conservative,Liberal,Liberal_perc,Conservative_perc,lift_liberal,lift_conservative,log_odds_liberal,log_odds_conservative
PoliticalViews,,,,,,,,
Conservative,47.600423,359.241015,5.610832,5.435391,3.339371,5.529559,-0.584561,0.584561
Very Conservative,32.666667,252.912281,3.957890,3.789924,3.533155,4.836270,-0.129836,0.129836
Moderate,45.112855,386.093543,6.029193,5.188620,4.897952,4.036384,0.428539,-0.428539
Apathetic,34.727528,325.516854,5.086309,4.042423,4.949048,3.214067,0.744521,-0.744521
Other,31.282667,290.936000,4.551824,3.679487,5.164304,3.240910,0.775231,-0.775231
Libertarian,41.478788,378.272727,5.915587,4.851722,6.126030,3.983378,0.886232,-0.886232
Liberal,42.996204,409.032895,6.388581,4.992468,6.363302,3.293671,1.221079,-1.221079
Very Liberal,38.820988,388.561728,6.075759,4.574150,7.228219,3.348357,1.519284,-1.519284


In [37]:
df_simplify=df_avg['log_odds_liberal']
df_simplify

PoliticalViews
Conservative        -0.584561
Very Conservative   -0.129836
Moderate             0.428539
Apathetic            0.744521
Other                0.775231
Libertarian          0.886232
Liberal              1.221079
Very Liberal         1.519284
Name: log_odds_liberal, dtype: float64